In [1]:
%matplotlib inline

In [2]:
import numpy as np
import os
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from math import sqrt
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
import time

def color(i,j):
    delta=i/j
    return (delta,delta,delta)

def plotImportantFeatures(matrix):
    matrixFeaturesImportances=[] 
    x = []
    n=0
    fig = plt.figure(2)
    for i in matrix :
        matrixFeaturesImportances.append(i.feature_importances_.tolist())
    nbFeatures =  len(matrixFeaturesImportances[0])
    #moyenne d'importance des features pour tous les classifieurs
    mean=[]
    sum=0
    for j in range(0,nbFeatures):
        sum=0
        for classifier in matrixFeaturesImportances :
            sum=sum + classifier[j]
        mean.append(sum/len(matrixFeaturesImportances))   
        
    for i in range(0,len(matrix)):
        e=[]
        for i in range(0,nbFeatures):
            n=n+1/nbFeatures
            e.append(n)
        x.append(e)
    width = 0.125
    plt.subplot(221)
    plt.title('feature importances on every classifier')
    plt.xlabel('features')
    for i in range(0,len(x)):
        colori=color(i%nbFeatures,nbFeatures)
        plt.bar(x[i],matrixFeaturesImportances[i], width, color=colori)
    plt.subplot(222)
    x= range(0,nbFeatures)
    plt.bar(x,mean, 1, color='b')
    plt.title('means of feature importances on every classifier')
    plt.xlabel('features')
    plt.show()
   
def trainClassifier(folderPath,maxTurn):
    classifier=[]
    classifierImportantFeatures=[]
    nbrTurnMax = len(os.listdir(folderPath))
    dataset = np.loadtxt(folderPath+'/turn'+str(1)+'.txt', delimiter=",")
    nbFeatures=len(dataset[0])
    for i in range(0,min(maxTurn,nbrTurnMax-1)):
        dataset = np.loadtxt(folderPath+'/turn'+str(i+1)+'.txt', delimiter=",")
        X = dataset[:,1:]
        y = dataset[:,0]
        model = LogisticRegression()
   
        forest = ExtraTreesClassifier()
        model.fit(X, y)
        forest.fit(X,y)
        classifier.append(model)
        classifierImportantFeatures.append(forest)
  
    plotImportantFeatures(classifierImportantFeatures)
    return classifier

def testClassifier(listClassifier,folderPathTest,maxTurn):

    listTestGame = os.listdir(folderPathTest)
    nbGameTestMax =len(listTestGame)
    resA=[]
    resB=[]
    listClassifier_pred_y=[]
    listClassifier_test_y=[]
    game = np.loadtxt(folderPathTest+'/game'+str(1)+'.txt', delimiter=",")

    for i in range(1,nbGameTestMax+1):
        game = np.loadtxt(folderPathTest+'/game'+str(i)+'.txt', delimiter=",")
        resGameA=[]
        resGameB=[]
        for j in range(0,min(len(game),min(maxTurn,len(listClassifier)))):
            if len(listClassifier_pred_y)<min(len(game),min(maxTurn+1,len(listClassifier))):
                listClassifier_pred_y.append([])
                listClassifier_test_y.append([])
            gameCour=game[:,1:]
            proba = listClassifier[j].predict_proba(gameCour[j].reshape(1, -1))[0][1]
            y = listClassifier[j].predict(gameCour[j].reshape(1, -1))
            listClassifier_pred_y[j].append(y[0])
            listClassifier_test_y[j].append(game[0][0])
            if game[0][0] == 1: #if "Me" won
                resGameA.append(proba)
            else :
                resGameB.append(1-proba)
          
        if len(resGameA)!=0:
            resA.append(resGameA)
        if len(resGameB)!=0:
            resB.append(resGameB)
        
    return resA,resB,listClassifier_pred_y,listClassifier_test_y

def plotProbaMeans(matrixProbaA,matrixProbaB):
    plt.figure(1)
    meanA=[]
    meanB=[]
    matrixALength=len(matrixProbaA)
    matrixBLength=len(matrixProbaB)
    for i in range(0, matrixALength):
        sum=0
        nbGameWithiTurn=0
        for j in matrixProbaA:
             if(len(j)>i):
                sum = sum + j[i]
                nbGameWithiTurn=nbGameWithiTurn+1
        if  nbGameWithiTurn>0:
            meanA.append(sum/ nbGameWithiTurn)
    for i in range(0, matrixBLength):
        sum=0
        nbGameWithiTurn=0
        for j in matrixProbaB:
             
            if(len(j)>i):
                sum = sum + j[i]
                nbGameWithiTurn=nbGameWithiTurn+1
        if  nbGameWithiTurn>0:
            meanB.append(sum/nbGameWithiTurn)
    
    plt.subplot(221)
    plt.plot(meanA)
    plt.ylim(0,1)
    plt.ylabel('mean P of predicted Victory for Me and Me wins')
    plt.xlabel('turns')
    plt.subplot(222)
    plt.plot(meanB)
    plt.ylim(0,1)
    plt.ylabel('mean P of predicted Victory for Opp and Opp wins')
    plt.xlabel('turns')
    plt.show()
    
def plotProba(matrixProbaA,matrixProbaB):
    plt.figure(1)
    for i in range(0,20):
        plt.subplot(221)
        plt.plot(matrixProbaA[i])
    for i in range (0,20):
        plt.subplot(222)
        plt.plot(matrixProbaB[i])
    plt.subplot(221)
    plt.ylim(0,1)
    plt.ylabel('P of predicted Victory for Me and Me wins')
    plt.xlabel('turns')
    plt.subplot(222)
    plt.ylim(0,1)
    plt.ylabel('P of predicted Victory for Opp and Opp wins')
    plt.xlabel('turns')
    plt.show()
      
def printScore(listClassifier_pred_y,listClassifier_test_y):
    for i in range(0,len(listClassifier_pred_y)):
        print('classifier : '+str(i+1))
        print('f1_score : '+str(f1_score(listClassifier_test_y[i],listClassifier_pred_y[i],average='binary')))
        print('MCC :'+str( matthews_corrcoef(listClassifier_test_y[i],listClassifier_pred_y[i]))+'\n')

In [3]:
t1 = time.time()
#DataSets 1 tour = 2 players play
nbTurn=9 # nb last turn to be considered

listClassifier=trainClassifier('partiesHS/Datasets_GameOf9Turn/Datasets_train_1turn2plays',nbTurn)
matrixProbaA,matrixProbaB,listClassifier_pred_y,listClassifier_test_y=testClassifier(listClassifier,'partiesHS/Datasets_GameOf9Turn/Datasets_test_1turn2plays',nbTurn)
printScore(listClassifier_pred_y,listClassifier_test_y)
plotProba(matrixProbaA,matrixProbaB)
plotProbaMeans(matrixProbaA,matrixProbaB)

t2 = time.time() - t1
print (t2)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'partiesHS/Datasets_GameOf9Turn/Datasets_train_1turn2plays'

In [ ]:
#DataSets 1 turn = 1 player plays
#listClassifier=trainClassifier('partiesHS/Datasets_train')
#matrixProbaA,matrixProbaB,listClassifier_pred_y,listClassifier_test_y=testClassifier(listClassifier,'partiesHS/Datasets_test','partiesHS')
#printScore(listClassifier_pred_y,listClassifier_test_y)
#plotProbaMeans(matrixProbaA,matrixProbaB)